In [3]:
# Here is the imports
from tensorflow import keras
import numpy as np
#import matplotlib.pylot as plt
import tensorflow as tf
import segmentation_models as sm
sm.set_framework('tf.keras')
# segmentation_models could also use `tf.keras` if you do not have Keras installed
# or you could switch to other framework using `sm.set_framework('tf.keras')`

<h1>CONSTANTS</h1>

In [ ]:
# Dataset Constants
DATASET_PATH = ".\\dataset\\"
TRAIN_DIR = "train"
VAL_DIR = "validation"
TEST_DIR = "test"

DATA_DIR = "data"
LABEL_DIR = "label"

# Model Constants
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 2
CLASSES = ['kanama', 'iskemik']
LR = 0.0001
EPOCHS = 40
MODEL_SAVE_PATH = "./models"

In [ ]:
# Variables
x_train_dir = os.path.join(DATASET_PATH, TRAIN_DIR, DATA_DIR)
y_train_dir = os.path.join(DATASET_PATH, TRAIN_DIR, LABEL_DIR)

x_val_dir = os.path.join(DATASET_PATH, VAL_DIR, DATA_DIR)
y_val_dir = os.path.join(DATASET_PATH, VAL_DIR, LABEL_DIR)

x_test_dir = os.path.join(DATASET_PATH, TEST_DIR, DATA_DIR)
y_test_dir = os.path.join(DATASET_PATH, TEST_DIR, LABEL_DIR)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint('./best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]

In [ ]:
def preprocessing() -> tf.data.Dataset:
    preprocess_input = sm.get_preprocessing(BACKBONE)
    pass

In [ ]:
def model_factory() -> keras.Model:
    # define network parameters
    n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
    activation = 'sigmoid' if n_classes == 1 else 'softmax'

    #create model
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

    # define optomizer
    optim = keras.optimizers.Adam(LR)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    # set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
    # TODO redefine class weights
    dice_loss = sm.losses.DiceLoss(class_weights=np.array([1, 2, 0.5])) 
    focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
    total_loss = dice_loss + (1 * focal_loss)

    # actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
    # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

    # compile keras model with defined optimozer, loss and metrics
    model.compile(optim, total_loss, metrics)
    return model

In [ ]:
def train_model(data_generators, model, callbacks):
    train_dataloader = data_generators[0]
    valid_dataloader = data_generators[1]
    # train model
    history = model.fit(
        train_dataloader, 
        steps_per_epoch=len(train_dataloader), 
        epochs=EPOCHS, 
        callbacks=callbacks, 
        validation_data=valid_dataloader, 
        validation_steps=len(valid_dataloader),
    )
    # TODO make this after eval for writing accuracy in the model name
    save_path = os.path.join(MODEL_SAVE_PATH, f"{}acc_{}.{}")
    model.save(save_path)

In [ ]:
def evaluate_model(eval_generator, model):
    pass

In [ ]:
def plot_history(train_history):
    # Plot training & validation iou_score values
    plt.figure(figsize=(30, 5))
    plt.subplot(121)
    plt.plot(train_history.history['iou_score'])
    plt.plot(train_history.history['val_iou_score'])
    plt.title('Model iou_score')
    plt.ylabel('iou_score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(train_history.history['loss'])
    plt.plot(train_history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# TODO Main code comes here
generators = preprocessing()
model = model_factory()


In [8]:
from tensorflow.keras.applications import VGG19
model = VGG19(include_top=False, weights='imagenet', input_tensor=keras.layers.Input(shape=(None, None, 3)))
#output = model.get_layer("block5_conv4").output

In [41]:
model.layers[-1]

In [42]:
model.output

<KerasTensor: shape=(None, None, None, 512) dtype=float32 (created by layer 'block5_pool')>

In [48]:
model.layers[-2].output

<KerasTensor: shape=(None, None, None, 512) dtype=float32 (created by layer 'block5_conv4')>

In [4]:
backbone = sm.Backbones.get_backbone("vgg19", include_top=False, weights='imagenet', input_shape=(None, None, 3))

In [6]:
backbone.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [11]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     